In [1]:
import pandas as pd
import numpy as np
import igraph as ig
import copy
import random

In [9]:
from logistica.ruteo import Ruteo
from logistica.utils import preparar_df_pedidos

In [7]:
df_clientes = pd.read_excel("data/inputs/data_inputs.xlsx", sheet_name="clientes")
df_pedidos = pd.read_excel("data/inputs/data_inputs.xlsx", sheet_name="pedidos")
df_pedidos = pd.merge(left=df_pedidos, right=df_clientes, on="cliente", how="inner")
df_pedidos

,cliente,pedidos1,pedidos2,pedidos3,pedidos4,pedidos5,pedidos6,coord_x,coord_y
0,A,4,3,5,2,6,6,0.257571,1.803726
1,B,6,6,5,4,1,3,1.523313,2.102301
2,C,1,6,6,2,6,7,0.710910,2.629800
3,D,8,6,5,2,8,2,1.013940,2.149740
4,E,4,1,2,4,2,3,0.609480,0.666810
5,F,3,1,6,8,6,5,0.998640,1.451790
6,G,3,2,3,7,4,3,1.434510,0.018180
7,H,5,8,5,2,4,8,2.171880,0.764910
8,I,5,2,3,4,2,6,1.412280,0.863190
9,J,1,8,6,8,1,2,2.386260,1.387260


In [10]:
df_pedido1 = preparar_df_pedidos(df_pedidos, "pedidos5")
df_pedido1.head()

,cliente,pedidos,coord_x,coord_y
0,A,6,0.257571,1.803726
1,B,1,1.523313,2.102301
2,C,6,0.710910,2.629800
3,D,8,1.013940,2.149740
4,E,2,0.609480,0.666810


In [11]:
df_camiones = pd.read_excel("data/inputs/data_inputs.xlsx", sheet_name="camiones")
df_camiones

,camion,carga_max,pedidos_max,dist_max
0,1,12,3,2
1,2,12,3,2
2,3,12,3,2
3,4,12,3,2
4,5,12,3,2
5,6,12,3,2


In [12]:
ruteo = Ruteo(df_camiones, df_pedido1, 3000, 1220)

In [13]:
ruteo.get_solucion_inicial("random", random_state=65)

In [22]:
#[[pedido.x, pedido.y, pedido.carga, pedido.camion_ix if pedido.asignado else None] for pedido in ruteo.get_pedidos()]

In [ ]:
# Las aristas serán las uniones entre pedidos que pueden estar juntos.
edges = [()]

In [5]:
locations = [(x, y) for x, y in zip(df_pedido1.coord_x, df_pedido1.coord_y)]
print(locations)
g1 = ig.Graph(locations)

In [28]:
import plotly.express as px
import plotly.graph_objects as go

In [43]:
node_ix = []
node_x = []
node_y = []
node_carga = []
node_ix_camion = []

for pedido in ruteo.get_pedidos():
    node_ix.append(pedido.ix)
    node_x.append(pedido.x)
    node_y.append(pedido.y)
    node_carga.append(pedido.carga)
    node_ix_camion.append(str(pedido.camion_ix) if pedido.asignado else "No Asignado")

# node_ix = [pedido.ix for pedido in ruteo.get_pedidos()]
# node_x = [pedido.x for pedido in ruteo.get_pedidos()]
# node_y = [pedido.y for pedido in ruteo.get_pedidos()]
# node_carga = [pedido.carga for pedido in ruteo.get_pedidos()]
# node_ix_camion = [str(pedido.camion_ix) if pedido.asignado else "None" for pedido in ruteo.get_pedidos()]

fig = px.scatter(x=node_x,
                 y=node_y,
                 size=node_carga,
                 color=node_ix_camion,
                 text=node_ix,
                 height=600,
                 width=700,
                 template="plotly_white")

#fig.add_trace(go.Scatter)

fig.show()

In [ ]:
G = ig.

In [ ]:
edge_x = []
edge_y = []
for edge in G.edges():
    x0, y0 = G.nodes[edge[0]]['pos']
    x1, y1 = G.nodes[edge[1]]['pos']
    edge_x.append(x0)
    edge_x.append(x1)
    edge_x.append(None)
    edge_y.append(y0)
    edge_y.append(y1)
    edge_y.append(None)

edge_trace = go.Scatter(
    x=edge_x, y=edge_y,
    line=dict(width=0.5, color='#888'),
    hoverinfo='none',
    mode='lines')

node_x = []
node_y = []
for node in G.nodes():
    x, y = G.nodes[node]['pos']
    node_x.append(x)
    node_y.append(y)

node_trace = go.Scatter(
    x=node_x, y=node_y,
    mode='markers',
    hoverinfo='text',
    marker=dict(
        showscale=True,
        # colorscale options
        #'Greys' | 'YlGnBu' | 'Greens' | 'YlOrRd' | 'Bluered' | 'RdBu' |
        #'Reds' | 'Blues' | 'Picnic' | 'Rainbow' | 'Portland' | 'Jet' |
        #'Hot' | 'Blackbody' | 'Earth' | 'Electric' | 'Viridis' |
        colorscale='YlGnBu',
        reversescale=True,
        color=[],
        size=10,
        colorbar=dict(
            thickness=15,
            title='Node Connections',
            xanchor='left',
            titleside='right'
        ),
        line_width=2))